In [6]:
# imports

import requests
import json
import pandas as pd
import csv
from time import sleep


### Toronto's (Bixi-Toronto) bike stations.

In [2]:
# READ THE FULL LIST OF CITY BIKE STATIONS FROM CSV
city_bikes_df = pd.read_csv("../csv_folder/city_bikes_df.csv")

### NOT EXECUTED - Shrink the radius of city bike stations to latitude 43.64

In [ ]:
# FILTER DOWN CITY BIKE DATA FRAME TO LATITUDE 43.46

# city_bikes_downtown_df = city_bikes_df[city_bikes_df['latitude'].astype(str).str.contains("43.64")]


# UPLOAD THE 43.64 DATAFRAME TO CSV
# city_bikes_downtown_df.to_csv("../csv_folder/city_bikes_downtown_df.csv", index=False)

In [1]:
# # CHECK FOR NULLS
# city_bikes_downtown_df.isnull().sum()

In [11]:
city_bikes_df = pd.read_csv("../csv_folder/city_bikes_df.csv", index_col=0)
city_bikes_df

,name,latitude,longitude,timestamp,free_bikes,empty_slots,renting,returning,ebikes
0,Summerhill Ave / Maclennan Ave,43.685924,-79.376304,2025-07-28T22:39:27.286065+00:00Z,4,4,1,1,1
1,Queen St E / Joseph Duggan Rd,43.667763,-79.308117,2025-07-28T22:39:27.288543+00:00Z,16,1,1,1,5
2,Victoria Park Ave / Danforth Ave,43.691468,-79.288619,2025-07-28T22:39:27.287666+00:00Z,2,16,1,1,0
3,Navy Wharf Crt / Bremner Blvd,43.640722,-79.391051,2025-07-28T22:39:27.134271+00:00Z,1,5,1,1,0
4,420 Wellington St W,43.643834,-79.396649,2025-07-28T22:39:27.135699+00:00Z,2,6,1,1,0
...,...,...,...,...,...,...,...,...,...
932,457 King St W,43.645209,-79.396074,2025-07-28T22:39:27.185775+00:00Z,7,10,1,1,0
933,Danforth Ave / Coxwell Ave,43.683378,-79.322961,2025-07-28T22:39:27.134872+00:00Z,15,0,1,1,6
934,Bloor St W / Brock Ave,43.658988,-79.438715,2025-07-28T22:39:27.288153+00:00Z,5,5,1,1,2
935,Wells Hill Ave / St. Clair Ave W,43.683351,-79.415620,2025-07-28T22:39:27.186412+00:00Z,1,14,1,1,1


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:

# CATEGORIES = PARK, BAR, RESTAURANT
categories = "4bf58dd8d48988d163941735,4bf58dd8d48988d116941735,4d4b7105d754a06374d81259"


#FOURSQUARE API KEY FROM FILE
with open("/Users/lhl/Documents/Stuff/foursquare_key.txt") as f:
    FOURSQUARE_KEY = f.read().strip()

headers = {
    "Accept": "application/json;version=2023-08-01",
    "Authorization": f"Bearer {FOURSQUARE_KEY}",
    "X-Places-Api-Version": "2025-06-17"
}

# HELPER FUNCTION TO CALL API URL
def get_venues_fs(latitude, longitude, radius, categories):
    return f"https://places-api.foursquare.com/places/search?ll={latitude},{longitude}&radius={radius}&fsq_category_ids={categories}"

# STORE RESULTS IN A LIST
fsqr_results = []

# FOR EACH ROW IN CITYBIKE DATAFRAME, MAKE A CALL TO FOURSQUARE AND GATHER POI DETAILS
for i, row in city_bikes_df.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    station_id = row.name # INDEX

    url = get_venues_fs(lat, lon, 1000, categories)  # USE THE FUNCTION FROM ABOVE
    response = requests.get(url, headers=headers)

    if response.status_code == 200:   # CHECK FOR SUCCESSFUL CONNECTION
        data = response.json()
        for place in data.get('results', []):
            fsqr_results.append({    # POI DETAILS
                'station_id': station_id,
                'place_name': place['name'],
                'latitude': place['latitude'],
                'longitude': place['longitude'],
                'address': place['location'].get('formatted_address', ''),
                'categories': [c['name'] for c in place.get('categories', [])]
            })
    else:
        print(f"Error at index {station_id}: {response.status_code}")

Put your parsed results into a DataFrame

In [ ]:
# STORE RESULTS IN DATAFRAME
fsqr_df = pd.DataFrame(fsqr_results)

# UPLOAD DATAFRAME TO CSV
fsqr_df.to_csv("../csv_folder/fsqr_response_df.csv")

In [14]:
fsqr_df.head()
fsqr_df['categories'].explode().value_counts()

categories
Park                    1253
Bar                      693
Restaurant               686
Fast Food Restaurant     551
Italian Restaurant       481
                        ... 
Mac and Cheese Joint       1
Piano Bar                  1
Soccer Field               1
Syrian Restaurant          1
Empanada Restaurant        1
Name: count, Length: 205, dtype: int64

# Yelp

# AS YELP REQUIRES A BUSINESS EMAIL FOR TRIAL SUBSCRIPTION, THIS WAS NOT EXCECUTED 

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating